In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from scipy.integrate import solve_ivp
from scipy import optimize
import random
import networkx as nx

In [ ]:
def start(nodes, sickPerc, radius):
    Network = nx.random_geometric_graph(nodes, radius)
    Situation = []
    for i in range(nodes):
        randomN = random.uniform(0, 1)
        if randomN < sickPerc:
            Situation.append(1)
        else:
            Situation.append(0)
    return Network, Situation

def updateSIS(y, beta, gamma, Network):
    yNew = []
    for i in range(len(Network)):
        if y[i] == 0:
            Connected = Network[i]
            Phealthy = 1
            for j in Connected:
                if y[j] == 1:
                    Phealthy *= (1 - beta)
            PSick = 1 - Phealthy
            randomN = random.uniform(0, 1)
            if randomN < PSick:
                yNew.append(1)
            else:
                yNew.append(y[i])
        else:
            randomN = random.uniform(0, 1)
            if randomN < gamma:
                yNew.append(0)
            else:
                yNew.append(y[i])
    return yNew

def spreaddiseaseSIS(NLoops, StartSituation, beta, gamma, Network):
    yList = [StartSituation]
    yNew = StartSituation
    for i in range(NLoops):
        y = yNew
        yNew = updateSIS(y, beta, gamma, Network)
        yList.append(yNew)
    return yList

def countingSIS(situations):
    S = []
    I = []
    for i in range(len(situations)):
        S.append(np.bincount(situations[i], minlength = 2)[0])
        I.append(np.bincount(situations[i], minlength = 2)[1])
    return S, I

def everythingSIS(nodes, sickPerc, radius, NLoops, beta, gamma, color):
    Network, StartSituation = start(nodes, sickPerc, radius)
    situations = spreaddiseaseSIS(NLoops, StartSituation, beta, gamma, Network)
    S, I = countingSIS(situations)
    time = np.arange(0, NLoops + 1, 1)
    Fig_Data = plt.figure()

    plt.plot(time, S, label = 'S', color= color[0])
    plt.plot(time, I, label = 'I', color= color[1])

    plt.title('', size= 20)
    plt.ylabel('', size= 20)
    plt.xlabel('', size= 20)
    plt.legend(fontsize= 12)
    plt.tick_params(axis= 'both', which= 'major', labelsize= 14)
    plt.grid()

    plt.show()
    return Network, S, I

def averageSIS(nodes, sickPerc, radius, NLoops, beta, gamma, N, color):
    time = np.arange(0, NLoops + 1, 1)
    AverageS = np.zeros(NLoops + 1)
    AverageI = np.zeros(NLoops + 1)
    average_connected = 0
    for i in range(N):
        Network, StartSituation = start(nodes, sickPerc, radius)
        situations = spreaddiseaseSIS(NLoops, StartSituation, beta, gamma, Network)
        S, I = countingSIS(situations)
        AverageS += S
        AverageI += I
        # gemiddeld aantal connecties per node berekenen
        average_connected_temp = 0
        for j in range(len(Network)):
            average_connected_temp += len(Network[j])
        average_connected += average_connected_temp/len(Network)
    average_connected = average_connected/N
    AverageS = AverageS/N
    AverageI = AverageI/N
    
    
    
    Fig_Data = plt.figure()

    plt.plot(time, AverageS, label = 'S', color= color[0])
    plt.plot(time, AverageI, label = 'I', color= color[1])

    plt.title('', size= 20)
    plt.ylabel('', size= 20)
    plt.xlabel('', size= 20)
    plt.legend(fontsize= 12)
    plt.tick_params(axis= 'both', which= 'major', labelsize= 14)
    plt.grid()
    
    plt.show()
    return AverageS, AverageI, average_connected

In [ ]:
def I_function(t, betaf, gammaf): 
    Iinfty = (1 - gammaf/betaf)*nodes
    Chi = betaf - gammaf
    V = Iinfty/(sickPerc*nodes - 1)
    return Iinfty/(1 + V*np.exp(-Chi*t))

# Beta: Het gemiddelde aantal mensen waarmee ge verbonden zijt, 
# vermenigvuldigt met de kans om iemand in contact met u te besmetten op 1 tijdstap 
# Gamma: Kans om te genezen op 1 tijdstap

In [ ]:
N = 100
nodes = 200
sickPerc = 0.1
radius = 0.12
beta = 0.1 # Kans om iemand in contact met een geïnfecteerde te besmetten op 1 tijdstap
gamma = 0.02 # Kans om te genezen op 1 tijdstap
NLoops = 50
color = ['palegreen', 'tomato']

AverageS, AverageI, average_connected = averageSIS(nodes, sickPerc, radius, NLoops, beta, gamma, N, color)

In [ ]:
print(average_connected) # Gemmiddelde aantal edges per node
print(nodes*np.pi*radius**2) # Wat wij denken dat het gemmiddelde aantal edges per node zou zijn (behalve voor de randen)
average_connected/(nodes*np.pi*radius**2)

In [ ]:
time = np.arange(0, NLoops + 1, 1)
par = optimize.curve_fit(I_function, time, AverageI, p0= [beta, gamma])
fig = plt.figure()
plt.plot(time, I_function(time, par[0][0], par[0][1]))
plt.plot(time, AverageI)
plt.grid()
plt.show()
R0 = par[0][0]/par[0][1]
print(R0, par[0][0], par[0][1]) # betaf = par[0][0]
print(beta*average_connected) # Wij dachten dat dit gelijk zou moeten zijn aan de gefitte betaf

In [ ]:
betaf = beta*average_connected
gammaf = gamma
time = np.arange(0, NLoops + 1, 1)
fig = plt.figure()
plt.plot(time, I_function(time, betaf, gammaf))
plt.plot(time, AverageI)
plt.grid()
plt.show()